In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import itertools
import json

Обозначим все узлы дорожной сети числами от 0 до 56, вокзал будет 0. Введем матрицу $57\times 57$ стоимости перемещения $A_{ij}$, равную стоимости перемещения между узлами с номерами $i,j$. Введем также вектор $B_j$, который соответствует числу людей в экскурсионной группе для узла $j$. За множество $S$ обозначим номера всех достопримечательностей.

In [394]:
MAX_INT = 100
nodes_frame = pd.read_csv("task-2-nodes.csv", header=None, index_col=0).sort_values(1)
edges_frame = pd.read_csv("task-2-adjacency_matrix.csv", index_col=0, na_values=["-"])

# Sorting nodes such that initial is 0, and all crossings are at the beginning
initial_node = "Вокзал"
node_names = nodes_frame.index.tolist()
node_names.remove(initial_node)
node_names.insert(0, initial_node)
assert len(node_names) == 57 and node_names[0]==initial_node, "Sonething goes wrong with reordering initial node"

A = edges_frame.reindex(index=node_names, columns=node_names).to_numpy(na_value=MAX_INT, copy=True).astype(int)
B = nodes_frame.reindex(index=node_names).to_numpy(na_value=MAX_INT, copy=True).astype(int).reshape((-1))
sights_start = np.searchsorted(B, 0, side='right')  # Start of nodes which should be visited only once

In [395]:
A

array([[  0, 100, 100, ..., 100, 100, 100],
       [100,   0,   5, ..., 100, 100, 100],
       [100,   5,   0, ..., 100, 100, 100],
       ...,
       [100, 100, 100, ...,   0, 100,   3],
       [100, 100, 100, ..., 100,   0, 100],
       [100, 100, 100, ...,   3, 100,   0]])

In [396]:
B.sum()

134

In [397]:
len(list(*np.where(B)))

35

Поскольку имеется ограничение в 15 тактов, заведем для каждого автобуса векторы как в отборочной задаче, однако допустим, чтобы вершины повторялись. Тогда нам потребуется отжиг $15 \cdot 14 \cdot 57 = 11970$ кубитов. Это довольно-таки много, и асимптотика также плохая.

По сути, минимально нам требуется закодировать маршрут каждого автобуса. Для этого достаточно указать 14 промежуточных вершин, подразумевая что 0-ая и 15-ая это вокзал. Как минимум нам требуется $15 \cdot 14 = 210$ дискретных переменных от 0 до 56. В лучшем случае, используя двочную кодировку нам потребуется $\lceil \log_2 57 \rceil = 6$ кубитов, что дает вменяемый размер задачи $15 \cdot 14 \cdot 6 = 1260$. Проблема может возникнуть на этапе вычисления выражений
$$
a_{v^b_i, v^b_j} = \sum_{\alpha, \beta} a_{\alpha, \beta}\delta(\alpha, v^b_i) \delta(\beta, v^b_j)
$$
Если $\alpha = \sum_{i=0}^{5} \alpha[i] 2^{i}$, а $v$ закодировано в виде 6 бит так же ($v = \sum_{i=0}^5 v[i]2^i$), имеем
$$
\delta(\alpha, v) = \prod_{i=0}^{5} (v[i]\cdot \alpha[i] + (1-v[i])\cdot(1 - \alpha[i]))
$$

## Прямолинейный метод кодирования

**Важно:** мы будем предполагать, что автобусу запрещено стоять на достопримечательности.

Заведем бинарный вектор $v^b_{ij}$, где $b\in\{0,\dots, 14\}$, $i\in \{1,\dots, 14\}$ и $j\in\{0,\dots, 57\}$, и
$$
v^b_{ij} = \begin{cases}
    1, & \text{если $b$-й автобус на $i$ такте перемещается в вершину $j$}, \\
    0, & \text{иначе}.
\end{cases}
$$

В таком случае имеются следующее естественное ограничение:
$$
\forall b, i \hookrightarrow \sum_{j} v^b_{ij} = 1,
$$
выражающее собой тот факт, что на $i$-м такте автобус находится хотя бы в одной вершине.

Его функция потерь есть
$$
L_v := \sum_{b,i} \left( \sum_j v^b_{ij} - 1 \right)^2 = \sum_{b,i} \left( \sum_{j,j'} v^b_{ij}v^b_{ij'} - 2 \sum_j v^b_{ij} + 1 \right)
$$

Далее определим условия в виде ограничений.
1. Автобусов уже 15. Чтобы не привысить вместимость, необходимо выполнить условие
$$
\forall b \hookrightarrow \sum_{j} B_j \sum_{i} v^b_{ij} \leq 10
$$
Вводим дополнительные переменные - число занятых мест в виде one-hot-encoding, $y^b_k = 1$ если в автобусе $b$ занято $k$ мест, $k=0,\dots,10$. Ограничения такой кодировки есть
$$
\forall b \hookrightarrow \sum_k y^b_k = 1,
$$
что соответствует потерям
$$
L_y := \sum_b \left( \sum_{k} y^b_k  - 1 \right)^2 = \sum_b \left( \sum_{k,k'} y^b_k y^b_{k'} - 2\sum_k y^b_k + 1 \right) .
$$
Тогда функция потерь этого пункта есть
$$
L_1 := \sum_b \left( \sum_{i,j} B_j v^{b}_{ij} - \sum_k k y^b_k \right)^2 = \sum_b \left( \sum_{i,j,i',j'} B_jB_{j'} v^{b}_{ij}v^{b}_{i'j'} + \sum_{k,k'} kk' y^b_k y^b_{k'} - 2\sum_{i,j,k} B_j k v^{b}_{ij} y^b_k \right)
$$

2. Достопримечательности должны быть посещены ровно 1 раз (и нельзя в них стоять)
$$
\forall j \in S \hookrightarrow \sum_{b,i} v^b_{ij} = 1
$$
Соответственно потери есть
$$
L_2 := \sum_{j\in S} \left(\sum_{b,i} v^b_{ij} - 1 \right)^2 = \sum_{j\in S} \left(\sum_{b,i,b',i'} v^{b}_{ij}v^{b'}_{i'j} - 2\sum_{b,i} v^b_{ij} + 1 \right)
$$

3. На узле (кроме вокзала) не более 1 машины
$$
\forall i, j \neq 0 \hookrightarrow \sum_{b} v^b_{i,j} \leq 1.
$$
Это эффективно соответствует функции потерь
$$
L_3 := \sum_{i,j\neq 0} \sum_{b_1\neq b_2} v^{b_1}_{ij} v^{b_2}_{ij}
$$

Минимизировать требуется функцию (старт и финиш на вокзале):
$$
H := \sum_{b} \left( \sum_j A_{0j}v^b_{1j} + \sum_{i=1}^{13} \sum_{j,j'} A_{jj'} v^b_{ij} v^b_{(i+1)j'} + \sum_j A_{j0} v^b_{14j} \right)
$$

Если мы хотим разрешить автобусу оставаться в пунктах с достопримечательностью, мы должны ввести дополнительную переменную.
Пусть автобус $b$ высаживает людей в такте $i$ в достопримечательность $j \in S$ если $z^b_{ij} = 1$. Имеем следующие ограничения:
$$
\forall j \hookrightarrow \sum_{i,b} z^b_{ij} = 1  
$$
(это - ограничение 2). Ограничение 1 превращается в
$$
\forall b \hookrightarrow \sum_{j} B_j \sum_{i} z^b_{ij} \leq 10
$$

In [398]:
NUM_BUSES = 15
MAX_PEOPLE = 10
MAX_TACTS = 15
NUM_TACTS = MAX_TACTS - 1
NUM_NODES = len(node_names)
NUM_PEOPLE = MAX_PEOPLE + 1

# Array will be encoded as {v[b,i,j], y[b,k]}
# b = 0,...,NUM_BUSES-1
# i = 1,...,NUM_TACTS-1
# j = 0,...,NUM_NODES-1
# k = 0,...,MAX_PEOPLE

def get_index(inp : list) -> int:
    """
    Returns integer index in big array of element v,b,i,j or y,b,k
    """
    var = inp
    if var[0] == "v":
        return (MAX_TACTS-1)*NUM_NODES*int(var[1]) + NUM_NODES*(int(var[2]) - 1) + int(var[3])
    elif var[0] == "y":
        return NUM_BUSES*NUM_TACTS*NUM_NODES + (MAX_PEOPLE + 1)*int(var[1]) + int(var[2])

In [399]:
SHAPE = NUM_BUSES*NUM_TACTS*NUM_NODES + NUM_BUSES*NUM_PEOPLE
H = np.zeros((SHAPE, SHAPE))
for b, j in itertools.product(range(NUM_BUSES), range(NUM_NODES)):
    ind = get_index(["v",b,1,j])
    H[ind, ind] += A[0,j]
    ind = get_index(["v",b,NUM_TACTS,j])
    H[ind, ind] += A[j,0]

for b, i, j, jp in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS-1), range(NUM_NODES), range(NUM_NODES)):
    H[get_index(["v",b,i,j]), get_index(["v",b,i+1,jp])] += A[j,jp]

In [400]:
Lv = np.zeros((SHAPE, SHAPE))
for b, i, j in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS), range(NUM_NODES)):
    Lv[get_index(["v",b,i,j]), get_index(["v",b,i,j])] -= 2
for b, i, j, jp in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS), range(NUM_NODES), range(NUM_NODES)):
    Lv[get_index(["v",b,i,j]), get_index(["v",b,i,jp])] += 1

In [401]:
Ly = np.zeros((SHAPE, SHAPE))
for b, k in itertools.product(range(NUM_BUSES), range(NUM_PEOPLE)):
    ind = get_index(["y", b, k])
    Ly[ind, ind] -= 2
    
for b, k, kp in itertools.product(range(NUM_BUSES), range(NUM_PEOPLE), range(NUM_PEOPLE)):
    Ly[get_index(["y", b, k]), get_index(["y", b, kp])] += 1

In [402]:
L1 = np.zeros((SHAPE, SHAPE))
for b in range(NUM_BUSES):
    for i,ip,j,jp in itertools.product(range(1,MAX_TACTS), range(1,MAX_TACTS), 
                                       range(sights_start, NUM_NODES), range(sights_start, NUM_NODES)):
        L1[get_index(["v", b, i, j]), get_index(["v", b, ip, jp])] += B[j]*B[jp]
    
    for k,kp in itertools.product(range(1, NUM_PEOPLE), range(1, NUM_PEOPLE)):
        L1[get_index(["y", b, k]), get_index(["y", b, kp])] += k*kp
        
    for i,j,k in itertools.product(range(1, MAX_TACTS), range(sights_start, NUM_NODES), range(1, NUM_PEOPLE)):
        L1[get_index(["v", b, i, j]), get_index(["y", b, k])] -= 2 * B[j] * k

In [403]:
L2 = np.zeros((SHAPE, SHAPE))
for j in range(sights_start, NUM_NODES):
    for b,i in itertools.product(range(NUM_BUSES), range(1, MAX_TACTS)):
        ind = get_index(["v", b, i, j])
        L2[ind, ind] -= 2
        for bp,ip in itertools.product(range(NUM_BUSES), range(1, MAX_TACTS)):
            ind2 = get_index(["v", bp, ip, j])
            L2[ind, ind2] += 1

In [404]:
L3 = np.zeros((SHAPE, SHAPE))
for i,j in itertools.product(range(1, MAX_TACTS), range(1, NUM_NODES)):
    for b,bp in itertools.combinations(range(NUM_BUSES), 2):
        L3[get_index(["v", b, i, j]), get_index(["v", bp, i, j])] += 1

In [405]:
NUM_SIGHTS = NUM_NODES - sights_start
Lv_ft = NUM_BUSES * NUM_TACTS
Ly_ft = NUM_BUSES
L2_ft = NUM_SIGHTS

In [406]:
#with open("Qmat.txt", mode='wt') as out_f:
    out_f.write(f"{SHAPE} {len(Q.row)}\n")
    for c,r,d in zip(Q.col, Q.row, Q.data):
        out_f.write(f"{r+1} {c+1} {d}\n")

IndentationError: unexpected indent (1632957687.py, line 2)

In [ ]:
# ARR_SHAPE_V = (NUM_BUSES, NUM_TACTS, NUM_NODES)
# SHAPE_V = NUM_BUSES*NUM_TACTS*NUM_NODES
# ARR_SHAPE_Y = (NUM_BUSES, NUM_PEOPLE)
# SHAPE_Y = NUM_BUSES*NUM_PEOPLE

# Hij = np.zeros((NUM_TACTS, NUM_NODES, NUM_TACTS, NUM_NODES))
# Hij[0,:,0,:] = np.diag(A[0,:])    # first term
# Hij[-1,:,-1,:] = np.diag(A[:,0])  # last term
# for i in range(NUM_TACTS - 1):
#     Hij[i,:,i+1,:] += A
# Ht = np.reshape(np.einsum("ab,ijkl->aijbkl", np.eye(NUM_BUSES), Hij), (SHAPE_V, SHAPE_V))

# np.reshape(Hij, (NUM_TACTS*NUM_NODES, NUM_TACTS*NUM_NODES))

In [409]:
def check_constraints(Lmat, vec, free_term=0):
    return vec.T @ Lmat @ vec + free_term

In [410]:
def print_routes(vec):
    for b in range(NUM_BUSES):
        print(f"Bus #{b} follows the next route:")
        route = []
        for i in range(1, MAX_TACTS):
            route.append(list(*np.where(vec[get_index(["v", b, i, 0]):get_index(["v", b, i+1, 0])])))
        print(route)
        print("It carries " + str(list(*np.where(vec[get_index(["y", b, 0]):get_index(["y", b, NUM_PEOPLE])]))) + " people")

In [505]:
np.where([0,0,0])[0].shape[0]

0

In [508]:
def get_routes(vec):
    routes = []
    for b in range(NUM_BUSES):
        route = []
        for i in range(1, MAX_TACTS):
            pos = np.where(vec[get_index(["v", b, i, 0]):get_index(["v", b, i+1, 0])])[0]
            if pos.shape[0] > 0:
                route.append(pos[0])
            elif len(route) > 0:
                route.append(route[-1])
            else:
                route.append(0)
        routes.append(route)
    return routes

In [509]:
from qctl.core.cloud_platform_client import CloudPlatformClient
from pathlib import Path
import time

In [413]:
CLOUD_PLATFORM_URL = "https://cloudos.qboard.tech"
WORKSPACE_ID = "workspace-85b32ae33c6340dcb1603d9a116f9b22"  # workspace should be already exists
IMAGE_NAME = "cloud-platform-quantum-modules-qboard"  # use correct image name

In [414]:
def print_status(msg: str):
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())} {msg}")

In [415]:
client = CloudPlatformClient(cloud_platform_url=CLOUD_PLATFORM_URL)
with open("logindata.json") as fld:
    logdat = json.load(fld)
client.login(username=logdat["login"], password=logdat["password"])

In [416]:
(10,10,1,10,10)

(10, 10, 1, 10, 10)

In [547]:
Pv = 250
Py = 250
P1 = 35
P2 = 190
P3 = 150

Pv = 520
Py = 520
P1 = 53
P2 = 150
P3 = 200

LvFT = Lv_ft*Pv
LyFT = Ly_ft*Py
L2FT = L2_ft*P2
FREE_TERM = LvFT + LyFT + L2FT
Pdict = {"Pv": Pv, "Py": Py, "P1": P1, "P2": P2, "P3": P3}

with open("weights_auto.json", mode="wt") as f_w_o:
    json.dump(Pdict, f_w_o)
    
client.put_file(source_path="weights_auto.json", dest_path="weights.json", workspace_id=WORKSPACE_ID)

{'path': '/weights.json',
 'size': 54,
 'nodeType': 'FILE',
 'lastModified': datetime.datetime(2024, 11, 15, 8, 39, 58),
 'contentType': 'application/json',
 'status': 'AWAITING_CHECK'}

In [548]:
process = client.create_process(
    workspace_id=WORKSPACE_ID,
    image=IMAGE_NAME,
    name=f"generating QUBO matrix",
    cpu="10",
    ram="16",
    gpu=0,
    command="python3",
    args=["/workspace/QUBO_formandrum_args.py", str(Pv), str(Py), str(P1), str(P2), str(P3), "500", "--out_file", f"Qmat.txt"],
)

print_status("Waiting for process to complete...")
while process["status"] != "COMPLETED":
    if process["status"] in ["CREATED", "SUBMITTED", "RUNNING", "COMPLETED"]:
        time.sleep(1)
    else:
        msg = client.get_process_output(process["id"])
        print_status("The process was not completed normally")
        raise ValueError(msg)
    process = client.describe_process(process["id"])
print_status("Process completed.")
process_output = client.get_process_output(process["id"])
print_status(f"Process_output: {process_output}")

2024-11-15 08:39:57 Waiting for process to complete...
2024-11-15 08:40:47 Process completed.
2024-11-15 08:40:48 Process_output: [{'timestamp': datetime.datetime(2024, 11, 15, 8, 40, 43, 818349), 'text': "['Вокзал', 'Канавинский мост 1', 'Перекресток 6', 'Перекресток 5', 'Мызинский мост 1', 'Перекресток 4', 'Перекресток 3', 'Перекресток 2', 'Перекресток 1', 'Перекресток 7', 'Неклюдово', 'Паромная переправа 2', 'Перекресток 8', 'Борский мост 1', 'Борский мост 2', 'Канавинский мост 2', 'Мызинский мост 2', 'Метромост 1', 'Метромост 2', 'Паромная переправа 1', 'Молитовский мост 1', 'Молитовский мост 2', 'Музей «Паровозы России»', 'Никольский собор', 'Нижне-Волжская набережная', 'Церковь Рождества Иоанна Предтечи на Торгу', 'Набережная Федоровского', 'Русский музей фотографии', 'Памятник Горькому', 'Мещерское озеро', 'Копосовская дубрава', 'Борский краеведческий музей', 'Борский музей военной техники', 'Спасский Староярмарочный собор', 'Литературный музей им. Горького', 'Рождественская цер

In [549]:
assert process_output[0]["text"] == str(node_names), "Different order!"

In [550]:
default_args = [
            "solve",
            "--matrix",
            f"/workspace/Qmat.txt",
            "--num-runs",
            "10",
            "--num-steps",
            "17000",
            "--output",
            f"/workspace/output.json",
            "--verbose",
            "1",
            "--gpu",
            "--dt",
            "0.5",
            "--seed",
            "42",
            "--target",
            f"{-FREE_TERM + 1000}",
            "--timeout",
            "300",
            "--convergence-tolerance",
            "0.000001",
            "--alpha",
            "0.9",
            "--sigma",
            "3.0",
            "--q",
            "0.1"
        ]

process = client.create_process(
    workspace_id=WORKSPACE_ID,
    image=IMAGE_NAME,
    name="run qiopt",
    cpu="10",
    ram="16",
    gpu=1,
    command="qiopt",
    args=default_args,
)

print_status("Waiting for process to complete...")
while process["status"] != "COMPLETED":
    if process["status"] in ["CREATED", "SUBMITTED", "RUNNING", "COMPLETED"]:
        time.sleep(1)
    else:
        msg = client.get_process_output(process["id"])
        print_status("The process was not completed normally")
        raise ValueError(msg)
    process = client.describe_process(process["id"])
print_status("Process completed.")
process_output = client.get_process_output(process["id"])
print_status(f"Process_output: {process_output}")

client.get_file("output.json", "output_auto.json", workspace_id=WORKSPACE_ID)

2024-11-15 08:40:48 Waiting for process to complete...
2024-11-15 08:41:43 Process completed.
2024-11-15 08:41:43 Process_output: [{'timestamp': datetime.datetime(2024, 11, 15, 8, 40, 52, 373945), 'text': '[info] : CUDA device: Quadro RTX 5000'}, {'timestamp': datetime.datetime(2024, 11, 15, 8, 40, 53, 219751), 'text': '[info] : Problem reading time: 824 milliseconds'}, {'timestamp': datetime.datetime(2024, 11, 15, 8, 40, 54, 102579), 'text': '[info] : SimCimSolverGpu::solve(): Running on 10 / 64 threads'}, {'timestamp': datetime.datetime(2024, 11, 15, 8, 41, 41, 675787), 'text': '[info] : Problem solving time: 48455 milliseconds'}, {'timestamp': datetime.datetime(2024, 11, 15, 8, 41, 41, 707282), 'text': '[info] : Result saved to /workspace/output.json file'}]


In [553]:
with open("output_auto.json", mode='rt') as in_f:
    res = json.load(in_f)
    
sol_vec = np.asarray(res["Solution"], dtype=int)
print("Objective: " + str(res["Objective"] + FREE_TERM))
print(f"Constraint Lv={check_constraints(Lv, sol_vec, Lv_ft)}")
print(f"Constraint Ly={check_constraints(Ly, sol_vec, Ly_ft)}")
print(f"Constraint L1={check_constraints(L1, sol_vec, 0)}")
print(f"Constraint L2={check_constraints(L2, sol_vec, L2_ft)}")
print(f"Constraint L3={check_constraints(L3, sol_vec, 0)}")

Objective: 10064
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=5.0
Constraint L2=16.0
Constraint L3=13.0


In [557]:
print_routes(sol_vec)

Bus #0 follows the next route:
[[6], [0], [0], [0], [14], [0], [0], [18], [0], [7], [22], [7], [21], [6]]
It carries [2] people
Bus #1 follows the next route:
[[21], [21], [0], [21], [7], [0], [0], [0], [15], [1], [2], [9], [34], [0]]
It carries [2] people
Bus #2 follows the next route:
[[0], [0], [6], [0], [6], [7], [8], [23], [23], [16], [8], [21], [0], [0]]
It carries [4] people
Bus #3 follows the next route:
[[0], [0], [18], [17], [2], [45], [1], [15], [18], [15], [1], [45], [1], [15]]
It carries [10] people
Bus #4 follows the next route:
[[0], [6], [39], [6], [5], [30], [5], [38], [9], [41], [12], [2], [0], [0]]
It carries [10] people
Bus #5 follows the next route:
[[49], [0], [0], [0], [15], [0], [18], [31], [32], [10], [13], [14], [0], [6]]
It carries [9] people
Bus #6 follows the next route:
[[0], [0], [26], [12], [36], [25], [24], [1], [17], [2], [3], [20], [20], [21]]
It carries [9] people
Bus #7 follows the next route:
[[15], [1], [2], [4], [9], [4], [3], [2], [1], [17], [18

In [558]:
rts = get_routes(sol_vec)
rts

[[6, 0, 0, 0, 14, 0, 0, 18, 0, 7, 22, 7, 21, 6],
 [21, 21, 0, 21, 7, 0, 0, 0, 15, 1, 2, 9, 34, 0],
 [0, 0, 6, 0, 6, 7, 8, 23, 23, 16, 8, 21, 0, 0],
 [0, 0, 18, 17, 2, 45, 1, 15, 18, 15, 1, 45, 1, 15],
 [0, 6, 39, 6, 5, 30, 5, 38, 9, 41, 12, 2, 0, 0],
 [49, 0, 0, 0, 15, 0, 18, 31, 32, 10, 13, 14, 0, 6],
 [0, 0, 26, 12, 36, 25, 24, 1, 17, 2, 3, 20, 20, 21],
 [15, 1, 2, 4, 9, 4, 3, 2, 1, 17, 18, 0, 7, 0],
 [0, 0, 21, 18, 18, 0, 15, 0, 6, 5, 6, 6, 14, 0],
 [42, 5, 5, 50, 7, 8, 16, 21, 0, 0, 21, 16, 21, 37],
 [18, 0, 0, 0, 21, 0, 0, 37, 0, 0, 0, 15, 0, 33],
 [14, 13, 13, 14, 0, 0, 6, 21, 21, 21, 20, 21, 0, 18],
 [14, 0, 15, 18, 0, 7, 21, 20, 2, 3, 2, 1, 15, 0],
 [0, 21, 16, 21, 0, 21, 7, 7, 7, 6, 0, 0, 0, 21],
 [15, 40, 27, 9, 12, 2, 2, 12, 14, 0, 29, 0, 0, 0]]

In [559]:
# Verification
# People limit
peoples = [sum([B[j] for j in r]) for r in rts]
print("Рассадка по автобусам:")
print(peoples)
if max(peoples) > 10:
    print("В некоторых автобусах более 10 людей")
    
# Single sights visiting
for j in range(sights_start, NUM_NODES):
    sj = sum([1 for r in rts for jp in r if jp == j])
    if sj > 1:
        print(f"Достопримечательность {node_names[j]} посещена более 1 раза")
    elif sj == 0:
        print(f"Достопримечательность {node_names[j]} не была посещена вообще")
        
# Single vechicle in tact
for i in range(NUM_TACTS):
    visited = set()
    for r in rts:
        if (r[i] in visited) and (r[i] != 0):
            print(f"Авария на такте {i}")
        visited.add(r[i])

# Total cost
cost = 0
for r in rts:
    for i in range(NUM_TACTS - 1):
        if A[r[i], r[i+1]] == MAX_INT:
            print(f"Внедорожное движение на такте {i}")
        cost += A[r[i], r[i+1]]
        
print(f"Стоимость всех поездок в сумме (включая штрафы за внедорожную езду) составила {cost}")

Рассадка по автобусам:
[2, 2, 4, 10, 11, 9, 9, 0, 0, 11, 5, 0, 0, 0, 7]
В некоторых автобусах более 10 людей
Достопримечательность Никольский собор посещена более 1 раза
Достопримечательность Памятник Горькому не была посещена вообще
Достопримечательность Рождественская церковь не была посещена вообще
Достопримечательность Нижегородский планетарий посещена более 1 раза
Достопримечательность Обзорная площадка на Гремячей горе не была посещена вообще
Достопримечательность Музей народов Нижегородского поволжья не была посещена вообще
Достопримечательность Благовещенский монастырь посещена более 1 раза
Достопримечательность Площадь Минина и Пожарского не была посещена вообще
Достопримечательность Большая Покровская улица не была посещена вообще
Достопримечательность Памятник Минину и Пожарскому не была посещена вообще
Достопримечательность Музей Сахарова не была посещена вообще
Достопримечательность Музей истории художественных промыслов не была посещена вообще
Достопримечательность Щёлоко

## Semiauto

In [562]:
scan_P = list(range(100, 150, 10))
processes = []
for i, p in enumerate(scan_P):
    proc = client.create_process(
        workspace_id=WORKSPACE_ID,
        image=IMAGE_NAME,
        name=f"generating QUBO matrix #{i}",
        cpu="10",
        ram="16",
        gpu=0,
        command="python3",
        args=["/workspace/QUBO_formandrum_args.py", str(Pv), str(Py), str(P1), str(p), str(P3), "500", "--out_file", f"Qmat#{i}.txt"],
    )
    processes.append(proc)

In [563]:
processes = []
for i, p in enumerate(scan_P):
    default_args = [
                "solve",
                "--matrix",
                f"/workspace/Qmat#{i}.txt",
                "--num-runs",
                "10",
                "--num-steps",
                "17000",
                "--output",
                f"/workspace/outputN{i}.json",
                "--verbose",
                "1",
                "--gpu",
                "--dt",
                "0.5",
                "--seed",
                "42",
                "--target",
                f"{-FREE_TERM + 1000}",
                "--timeout",
                "300",
                "--convergence-tolerance",
                "0.000001",
                "--alpha",
                "0.9",
                "--sigma",
                "3.0",
                "--q",
                "0.1"
            ]

    proc = client.create_process(
        workspace_id=WORKSPACE_ID,
        image=IMAGE_NAME,
        name=f"run qiopt #{i}",
        cpu="10",
        ram="16",
        gpu=1,
        command="qiopt",
        args=default_args,
    )
    processes.append(proc)

In [564]:
for i, p in enumerate(scan_P):
    print(i, p)
    client.get_file(f"outputN{i}.json", f"outputN{i}.json", workspace_id=WORKSPACE_ID)
    with open(f"outputN{i}.json", mode='rt') as in_f:
        res = json.load(in_f)

    sol_vec = np.asarray(res["Solution"], dtype=int)
    print(f"Constraint Lv={check_constraints(Lv, sol_vec, Lv_ft)}")
    print(f"Constraint Ly={check_constraints(Ly, sol_vec, Ly_ft)}")
    print(f"Constraint L1={check_constraints(L1, sol_vec, 0)}")
    print(f"Constraint L2={check_constraints(L2, sol_vec, L2_ft)}")
    print(f"Constraint L3={check_constraints(L3, sol_vec, 0)}")

0 100
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=8.0
Constraint L2=19.0
Constraint L3=6.0
1 110
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=11.0
Constraint L2=22.0
Constraint L3=9.0
2 120
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=22.0
Constraint L2=20.0
Constraint L3=9.0
3 130
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=7.0
Constraint L2=17.0
Constraint L3=9.0
4 140
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=17.0
Constraint L2=15.0
Constraint L3=10.0


In [303]:
processes = []
for i in range(10):
    default_args = [
                "solve",
                "--matrix",
                f"/workspace/Qmat.txt",
                "--num-runs",
                "10",
                "--num-steps",
                "15000",
                "--output",
                f"/workspace/outputN{i}.json",
                "--verbose",
                "1",
                "--gpu",
                "--dt",
                "0.5",
                "--seed",
                "42",
                "--target",
                f"{-FREE_TERM + 1000}",
                "--timeout",
                "300",
                "--convergence-tolerance",
                "0.000001",
                "--alpha",
                "0.9",
                "--sigma",
                "3.0",
                "--q",
                "0.1"
            ]

    proc = client.create_process(
        workspace_id=WORKSPACE_ID,
        image=IMAGE_NAME,
        name=f"run qiopt #{i}",
        cpu="10",
        ram="16",
        gpu=1,
        command="qiopt",
        args=default_args,
    )
    processes.append(proc)

In [305]:
for i in range(10):
    print(i)
    client.get_file(f"outputN{i}.json", f"outputN{i}.json", workspace_id=WORKSPACE_ID)
    with open(f"outputN{i}.json", mode='rt') as in_f:
        res = json.load(in_f)

    sol_vec = np.asarray(res["Solution"], dtype=int)
    print(f"Constraint Lv={check_constraints(Lv, sol_vec, Lv_ft)}")
    print(f"Constraint Ly={check_constraints(Ly, sol_vec, Ly_ft)}")
    print(f"Constraint L1={check_constraints(L1, sol_vec, 0)}")
    print(f"Constraint L2={check_constraints(L2, sol_vec, L2_ft)}")
    print(f"Constraint L3={check_constraints(L3, sol_vec, 0)}")

0
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=12.0
Constraint L2=13.0
Constraint L3=0.0
1
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=12.0
Constraint L2=13.0
Constraint L3=0.0
2
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=4.0
Constraint L2=7.0
Constraint L3=0.0
3
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=12.0
Constraint L2=13.0
Constraint L3=0.0
4
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=12.0
Constraint L2=13.0
Constraint L3=0.0
5
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=12.0
Constraint L2=13.0
Constraint L3=0.0
6
Constraint Lv=0.0
Constraint Ly=1.0
Constraint L1=12.0
Constraint L2=6.0
Constraint L3=1.0
7
Constraint Lv=0.0
Constraint Ly=1.0
Constraint L1=10.0
Constraint L2=6.0
Constraint L3=1.0
8
Constraint Lv=0.0
Constraint Ly=1.0
Constraint L1=12.0
Constraint L2=6.0
Constraint L3=1.0
9
Constraint Lv=0.0
Constraint Ly=0.0
Constraint L1=12.0
Constraint L2=13.0
Constraint L3=0.0


In [164]:
process = client.create_process(
    workspace_id=WORKSPACE_ID,
    image=IMAGE_NAME,
    name="generating QUBO matrix",
    cpu="10",
    ram="16",
    gpu=0,
    command="python3",
    args=["/workspace/QUBO_formandrum_args.py", str(Pv), str(Py), str(P1), str(P2), str(P3), "100", "--out_file", "QqqQmat.txt"],
)

print_status("Waiting for process to complete...")
while process["status"] != "COMPLETED":
    if process["status"] in ["CREATED", "SUBMITTED", "RUNNING", "COMPLETED"]:
        time.sleep(1)
    else:
        msg = client.get_process_output(process["id"])
        print_status("The process was not completed normally")
        raise ValueError(msg)
    process = client.describe_process(process["id"])
print_status("Process completed.")
process_output = client.get_process_output(process["id"])
print_status(f"Process_output: {process_output}")

2024-11-15 04:38:36 Waiting for process to complete...
2024-11-15 04:39:09 Process completed.
2024-11-15 04:39:09 Process_output: [{'timestamp': datetime.datetime(2024, 11, 15, 4, 39, 6, 341018), 'text': "['Вокзал', 'Канавинский мост 1', 'Перекресток 6', 'Перекресток 5', 'Мызинский мост 1', 'Перекресток 4', 'Перекресток 3', 'Перекресток 2', 'Перекресток 1', 'Перекресток 7', 'Неклюдово', 'Паромная переправа 2', 'Перекресток 8', 'Борский мост 1', 'Борский мост 2', 'Канавинский мост 2', 'Мызинский мост 2', 'Метромост 1', 'Метромост 2', 'Паромная переправа 1', 'Молитовский мост 1', 'Молитовский мост 2', 'Музей «Паровозы России»', 'Никольский собор', 'Нижне-Волжская набережная', 'Церковь Рождества Иоанна Предтечи на Торгу', 'Набережная Федоровского', 'Русский музей фотографии', 'Памятник Горькому', 'Мещерское озеро', 'Копосовская дубрава', 'Борский краеведческий музей', 'Борский музей военной техники', 'Спасский Староярмарочный собор', 'Литературный музей им. Горького', 'Рождественская церк

In [177]:
Q = sp.sparse.coo_array(Qnp)
with open("Qmat.txt", mode='wt') as out_f:
    out_f.write(f"{SHAPE} {len(Q.row)}\n")
    for c,r,d in zip(Q.col, Q.row, Q.data):
        out_f.write(f"{r+1} {c+1} {d}\n")

In [ ]:
client.put_file(source_path="Qmat.txt", dest_path="Qmat.txt", workspace_id=WORKSPACE_ID)

In [117]:
sol_vec @ Qnp @ sol_vec

-4816.0

In [118]:
res["Objective"]

-7130

## With Z variables

In [388]:
Pv = 35
Py = 35
P1 = 3.5
P2 = 25
P3 = 15

LvFT = Lv_ft*Pv
LyFT = Ly_ft*Py
L2FT = L2_ft*P2
FREE_TERM = LvFT + LyFT + L2FT

In [389]:
process = client.create_process(
        workspace_id=WORKSPACE_ID,
        image=IMAGE_NAME,
        name=f"generating QUBO matrix Z",
        cpu="10",
        ram="16",
        gpu=0,
        command="python3",
        args=["/workspace/QUBO_formandrum_args_z.py", str(Pv), str(Py), str(P1), str(P2), str(P3), "100", "--out_file", f"Qmat_z.txt"],
    )
print_status("Waiting for process to complete...")
while process["status"] != "COMPLETED":
    if process["status"] in ["CREATED", "SUBMITTED", "RUNNING", "COMPLETED"]:
        time.sleep(1)
    else:
        msg = client.get_process_output(process["id"])
        print_status("The process was not completed normally")
        raise ValueError(msg)
    process = client.describe_process(process["id"])
print_status("Process completed.")
process_output = client.get_process_output(process["id"])
print_status(f"Process_output: {process_output}")

2024-11-15 07:20:01 Waiting for process to complete...
2024-11-15 07:21:04 Process completed.
2024-11-15 07:21:04 Process_output: [{'timestamp': datetime.datetime(2024, 11, 15, 7, 21, 0, 266440), 'text': "['Вокзал', 'Канавинский мост 1', 'Перекресток 6', 'Перекресток 5', 'Мызинский мост 1', 'Перекресток 4', 'Перекресток 3', 'Перекресток 2', 'Перекресток 1', 'Перекресток 7', 'Неклюдово', 'Паромная переправа 2', 'Перекресток 8', 'Борский мост 1', 'Борский мост 2', 'Канавинский мост 2', 'Мызинский мост 2', 'Метромост 1', 'Метромост 2', 'Паромная переправа 1', 'Молитовский мост 1', 'Молитовский мост 2', 'Музей «Паровозы России»', 'Никольский собор', 'Нижне-Волжская набережная', 'Церковь Рождества Иоанна Предтечи на Торгу', 'Набережная Федоровского', 'Русский музей фотографии', 'Памятник Горькому', 'Мещерское озеро', 'Копосовская дубрава', 'Борский краеведческий музей', 'Борский музей военной техники', 'Спасский Староярмарочный собор', 'Литературный музей им. Горького', 'Рождественская церк

In [390]:
default_args = [
            "solve",
            "--matrix",
            f"/workspace/Qmat_z.txt",
            "--num-runs",
            "10",
            "--num-steps",
            "17000",
            "--output",
            f"/workspace/output.json",
            "--verbose",
            "1",
            "--gpu",
            "--dt",
            "0.5",
            "--seed",
            "42",
            "--target",
            f"{-FREE_TERM + 1000}",
            "--timeout",
            "300",
            "--convergence-tolerance",
            "0.000001",
            "--alpha",
            "0.9",
            "--sigma",
            "3.0",
            "--q",
            "0.1"
        ]

process = client.create_process(
    workspace_id=WORKSPACE_ID,
    image=IMAGE_NAME,
    name="run qiopt",
    cpu="10",
    ram="16",
    gpu=1,
    command="qiopt",
    args=default_args,
)

print_status("Waiting for process to complete...")
while process["status"] != "COMPLETED":
    if process["status"] in ["CREATED", "SUBMITTED", "RUNNING", "COMPLETED"]:
        time.sleep(1)
    else:
        msg = client.get_process_output(process["id"])
        print_status("The process was not completed normally")
        raise ValueError(msg)
    process = client.describe_process(process["id"])
print_status("Process completed.")
process_output = client.get_process_output(process["id"])
print_status(f"Process_output: {process_output}")

client.get_file("output.json", "output_auto.json", workspace_id=WORKSPACE_ID)

2024-11-15 07:21:05 Waiting for process to complete...
2024-11-15 07:22:08 Process completed.
2024-11-15 07:22:08 Process_output: [{'timestamp': datetime.datetime(2024, 11, 15, 7, 21, 9, 19251), 'text': '[info] : CUDA device: Quadro RTX 5000'}, {'timestamp': datetime.datetime(2024, 11, 15, 7, 21, 9, 882763), 'text': '[info] : Problem reading time: 842 milliseconds'}, {'timestamp': datetime.datetime(2024, 11, 15, 7, 21, 10, 805715), 'text': '[info] : SimCimSolverGpu::solve(): Running on 10 / 64 threads'}, {'timestamp': datetime.datetime(2024, 11, 15, 7, 22, 5, 316387), 'text': '[info] : Problem solving time: 55433 milliseconds'}, {'timestamp': datetime.datetime(2024, 11, 15, 7, 22, 5, 358805), 'text': '[info] : Result saved to /workspace/output.json file'}]


In [391]:
res["Objective"] + FREE_TERM

-65944

In [392]:
with open("output_auto.json", mode='rt') as in_f:
    res = json.load(in_f)
    
sol_vec = np.asarray(res["Solution"], dtype=int)
print(f"Constraint Lv={check_constraints(LvZ, sol_vec, Lv_ft)}")
print(f"Constraint Ly={check_constraints(LyZ, sol_vec, Ly_ft)}")
print(f"Constraint L1={check_constraints(L1Z, sol_vec, 0)}")
print(f"Constraint L2={check_constraints(L2Z, sol_vec, L2_ft)}")
print(f"Constraint L3={check_constraints(L3Z, sol_vec, 0)}")

Constraint Lv=210.0
Constraint Ly=15.0
Constraint L1=0.0
Constraint L2=35.0
Constraint L3=0.0


In [393]:
print_routes_z(sol_vec)

Bus #0 follows the next route:
[[29], [0], [], [41], [], [42], [], [18], [], [45], [], [39], [0], [49]]
Passengers exit in
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]
It carries [9] people
Bus #1 follows the next route:
[[0], [21], [], [25], [], [29], [], [44], [], [6], [], [9], [], [14]]
Passengers exit in
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]
It carries [8, 10] people
Bus #2 follows the next route:
[[39], [0], [], [30], [], [35], [], [50], [50], [], [29], [40], [49], [0]]
Passengers exit in
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]
It carries [9] people
Bus #3 follows the next route:
[[], [31], [], [48], [], [0], [18], [17], [], [25], [26], [26], [], [33]]
Passengers exit in
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]
It carries [10] people
Bus #4 follows the next route:
[[0], [15], [40], [], [47], [45], [45], [1], [24], [1], [43], [], [29], [0]]
Passengers exit in
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]
It 

In [384]:
rts, stps = get_routes_and_stops_z(sol_vec)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [382]:
peoples = [sum([B[j] for j in set(br)]) for br in rts]
peoples

[17, 19, 19, 14, 31, 15, 24, 28, 19, 16, 24, 27, 33, 29, 14]

In [344]:
def get_index_z(inp : list) -> int:
    """
    Returns integer index in big array of element v,b,i,j or y,b,k or z,b,i,j
    """
    var = inp
    if var[0] == "v":
        return NUM_TACTS*NUM_NODES*int(var[1]) + NUM_NODES*(int(var[2]) - 1) + int(var[3])
    elif var[0] == "y":
        return NUM_BUSES*NUM_TACTS*(NUM_NODES + NUM_SIGHTS) + (MAX_PEOPLE + 1)*int(var[1]) + int(var[2])
    elif var[0] == "z":
        return NUM_BUSES*NUM_TACTS*NUM_NODES + NUM_TACTS*NUM_SIGHTS*int(var[1]) + NUM_SIGHTS*(int(var[2]) - 1) + int(var[3]) - sights_start


In [385]:
def print_routes_z(vec):
    for b in range(NUM_BUSES):
        print(f"Bus #{b} follows the next route:")
        route = []
        stop = []
        for i in range(1, MAX_TACTS):
            route.append(list(*np.where(vec[get_index_z(["v", b, i, 0]):get_index_z(["v", b, i+1, 0])])))
            stop.append(list(*np.where(vec[get_index_z(["z", b, i, sights_start]):get_index_z(["v", b, i+1, sights_start])])))
        print(route)
        print(f"Passengers exit in")
        print(stop)
        print("It carries " + str(list(*np.where(vec[get_index_z(["y", b, 0]):get_index_z(["y", b, NUM_PEOPLE])]))) + " people")

In [383]:
def get_routes_and_stops_z(vec):
    routes = []
    stops = []
    for b in range(NUM_BUSES):
        route = []
        stop = []
        for i in range(1, MAX_TACTS):
            route.append(np.where(vec[get_index_z(["v", b, i, 0]):get_index_z(["v", b, i+1, 0])])[0][0])
            stop.append(np.where(vec[get_index_z(["z", b, i, sights_start]):get_index_z(["z", b, i+1, sights_start])])[0][0])
        routes.append(route)
        stops.append(stop)
    return (routes, stops)

In [352]:
SHAPEZ = NUM_BUSES*NUM_TACTS*(NUM_NODES + NUM_SIGHTS) + NUM_BUSES*NUM_PEOPLE

LvZ = np.zeros((SHAPEZ, SHAPEZ))
for b, i, j in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS), range(NUM_NODES)):
    Lv[get_index_z(["v",b,i,j]), get_index_z(["v",b,i,j])] -= 2
for b, i, j, jp in itertools.product(range(NUM_BUSES), range(1,MAX_TACTS), range(NUM_NODES), range(NUM_NODES)):
    Lv[get_index_z(["v",b,i,j]), get_index_z(["v",b,i,jp])] += 1

# Ly
print("Started Ly")
LyZ = np.zeros((SHAPEZ, SHAPEZ))
for b, k in itertools.product(range(NUM_BUSES), range(NUM_PEOPLE)):
    ind = get_index_z(["y", b, k])
    Ly[ind, ind] -= 2
    
for b, k, kp in itertools.product(range(NUM_BUSES), range(NUM_PEOPLE), range(NUM_PEOPLE)):
    Ly[get_index_z(["y", b, k]), get_index_z(["y", b, kp])] += 1

# L2 = Lz 
print("Started L2")
L2Z = np.zeros((SHAPEZ, SHAPEZ))
for j in range(sights_start, NUM_NODES):
    for b,i in itertools.product(range(NUM_BUSES), range(1, MAX_TACTS)):
        ind = get_index_z(["z", b, i, j])
        L2[ind, ind] -= 2
        for bp,ip in itertools.product(range(NUM_BUSES), range(1, MAX_TACTS)):
            ind2 = get_index_z(["z", bp, ip, j])
            L2[ind, ind2] += 1

# L1
print("Started L1")
L1Z = np.zeros((SHAPEZ, SHAPEZ))
for b in range(NUM_BUSES):
    for i,ip,j,jp in itertools.product(range(1,MAX_TACTS), range(1,MAX_TACTS), 
                                    range(sights_start, NUM_NODES), range(sights_start, NUM_NODES)):
        L1[get_index_z(["z", b, i, j]), get_index_z(["z", b, ip, jp])] += B[j]*B[jp]
    
    for k,kp in itertools.product(range(1, NUM_PEOPLE), range(1, NUM_PEOPLE)):
        L1[get_index_z(["y", b, k]), get_index_z(["y", b, kp])] += k*kp
        
    for i,j,k in itertools.product(range(1, MAX_TACTS), range(sights_start, NUM_NODES), range(1, NUM_PEOPLE)):
        L1[get_index_z(["z", b, i, j]), get_index_z(["y", b, k])] -= 2 * B[j] * k
        
# L3
print("Started L3")
L3Z = np.zeros((SHAPEZ, SHAPEZ))
for i,j in itertools.product(range(1, MAX_TACTS), range(1, NUM_NODES)):
    for b,bp in itertools.combinations(range(NUM_BUSES), 2):
        L3[get_index_z(["v", b, i, j]), get_index_z(["v", bp, i, j])] += 1


Started Ly
Started L2
Started L1
Started L3
